作業目標 : 實作 relu函數，並見識激活函數變動時的差別。

題目 : 執行一程式。把原本的 sigmoid函數更換成 relu函數 (提示:前五欄為更換完權重初始值的程式碼(為了方便比較 **主程式** 而附上)，所以作業從第六欄開始!)，並執行看結果。

In [ ]:
# 導入必要套件
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# 載入 MNIST 資料集
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1,)

# 區分輸入資料與正確解答(標籤)
x_org, y_org = mnist.data, mnist.target.astype(np.int)

In [ ]:
# step1 資料正規化, 令值的範圍在[0, 1]
x_norm = x_org / 255.0

# 在最前面加入虛擬變數 (1)
x_all=[]
h = np.array(x_norm)
for i in range(len(h)):
    x_all.append(np.insert(h[i],0,1))
x_all = np.array(x_all)

print('虛擬變數加入後的 shape', x_all.shape)

# step 2 轉換成 One-hot-Vector

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
y_all_one = ohe.fit_transform(np.c_[y_org])
print('One Hot Vector化後的 shape', y_all_one.shape)

# step 3 訓練資料、驗證資料分割

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test, y_train_one, y_test_one = train_test_split(
    x_all, y_org, y_all_one, train_size=60000, test_size=10000, shuffle=False)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape, 
    y_train_one.shape, y_test_one.shape)

In [ ]:
# 預測函數

# Sigmoid 函數
def sigmoid(x):
    return 1/(1+ np.exp(-x))

# Softmax 函數
def softmax(x):
    x = x.T
    x_max = x.max(axis=0)
    x = x - x_max
    w = np.exp(x)
    return (w / w.sum(axis=0)).T

# 交叉商函數
def cross_entropy(yt, yp):
    return -np.mean(np.sum(yt * np.log(yp), axis=1))

# 評估處理 (準確率與損失函數值)
from sklearn.metrics import accuracy_score

def evaluate(x_test, y_test, y_test_one, V, W):
    b1_test = np.insert(sigmoid(x_test @ V), 0, 1, axis=1)
    yp_test_one = softmax(b1_test @ W)
    yp_test = np.argmax(yp_test_one, axis=1)
    loss = cross_entropy(y_test_one, yp_test_one)
    score = accuracy_score(y_test, yp_test)
    return score, loss

In [ ]:
# mini batch 中取得 index 的類別 Index 

class Indexes():
    
    # 建構初始化
    def __init__(self, total, size):
        # 總共要產生多少個值
        self.total   = total
        # 每次要從 indexes 中取出幾個值
        self.size    = size
        # indexes 是個要放入資料的 tuple 或 list, 
        # 初始化一開始裏面是空的
        self.indexes = np.zeros(0) 

    # index取得関数    
    def next_index(self):
        # 預設為下次不需要再隨機產生新的 indexes 資料
        next_flag = False
        
    # 若 indexes 內剩餘的資料量小於 batch size, 
    # 則重新隨機產生資料放入 indexes 中, 且每筆資料不重複
        if len(self.indexes) < self.size: 
            self.indexes = np.random.choice(self.total, 
                self.total, replace=False)
            next_flag = True
            
        # 如果 indexes 中的資料量還足夠, 
        # 則直接取出 batch size 個 
        index = self.indexes[:self.size]
        self.indexes = self.indexes[self.size:]

        # 將取出的 batch size 筆資料傳回, 並顯示下次是否需要
        # 再重新隨機產生新的 indexes 
        return index, next_flag

In [ ]:
# 類別初始化
# 20: 總共有 20 筆資料
# 5: 每次取出 5 筆
indexes = Indexes(20, 5)

for i in range(6):
    # 呼叫 next_index 函數
    # 傳回值 1:  index 的 numpy 矩陣
    # 傳回值 2: 是否更新 index
    arr, flag = indexes.next_index()
    print(arr, flag)
    
    
# 隱藏層的節點數
H = 128
H1 = H + 1

# M: 訓練資料的筆數
M  = x_train.shape[0]

# D: 輸入資料的維數
D = x_train.shape[1]

# N: 分類的類別數
N = y_train_one.shape[1]

# mini-batch 梯度下降需要的參數設定
alpha = 0.01
nb_epoch = 100
batch_size = 512
B = batch_size

# 權重矩陣設定
V = np.ones((D, H))
W = np.ones((H1, N))

# 評估記錄 (損失函數值與準確率)
history2 = np.zeros((0, 3))

# mini-batch 將資料隨機產生索引, 並用 batch_size 取出
indexes = Indexes(M, batch_size)

# 記錄所有訓練資料用過幾輪的變數
epoch = 0

# 權重矩陣重新用 He normal 方法指定初值
V = np.random.randn(D, H) / np.sqrt(D / 2)
W = np.random.randn(H1, N) / np.sqrt(H1 / 2)
print(V[:2,:5])
print(W[:2,:5])


# 主程式

while epoch < nb_epoch:
    
    # 小批量取出訓練資料
    index, next_flag = indexes.next_index()
    x, yt = x_train[index], y_train_one[index]

    # 預測值計算 (前饋式) 
    a = x @ V                         
    b = sigmoid(a)                    # 激活函數為 sigmoid
    b1 = np.insert(b, 0, 1, axis=1)   # 增加虛擬變數 
    u = b1 @ W                         
    yp = softmax(u)                   
    
    # 誤差計算 
    yd = yp - yt                      
    bd = b * (1-b) * (yd @ W[1:].T)   

    # 梯度計算
    W = W - alpha * (b1.T @ yd) / B   
    V = V - alpha * (x.T @ bd) / B    

    if next_flag: # 1 epoch 結束後記錄
        score, loss = evaluate(
            x_test, y_test, y_test_one, V, W)
        history2 = np.vstack((history2, 
            np.array([epoch, loss, score])))
#        print("epoch = %d loss = %f score = %f" 
#             % (epoch, loss, score))
        epoch = epoch + 1

### 作業開始

Relu函數:
$$f(x) = \left\{\begin{matrix}
0, \,\,\,\,\,\,\,\, if \,\,\, x<0\\
x, \,\,\,\,\,\,\,\, if \,\,\, x>0.
\end{matrix}\right.$$

In [ ]:
# 從新給一次初始化的超參數
# 隱藏層節點數
H = 128
H1 = H + 1

# M: 訓練資料筆數
M  = x_train.shape[0]

# D: 輸入資料維數
D = x_train.shape[1]

# N: 分類的類別數
N = y_train_one.shape[1]

# 機器學習參數
alpha = 0.01
nb_epoch = 100
batch_size = 512
B = batch_size

# 權重矩陣初始化
V = np.random.randn(D, H) / np.sqrt(D / 2)
W = np.random.randn(H1, N) / np.sqrt(H1 / 2)

# 評估記錄用 (損失函數值與準確率)
history3 = np.zeros((0, 3))

# 為了 mini-batch 取樣做訓練資料隨機排序
indexes = Indexes(M, batch_size)

# 記錄總共執行多少輪的變數
epoch = 0

In [ ]:
# ReLU 函數
def ReLU(x):
    return np.maximum(0, x)

# step 函數, ReLu 的微分
def step(x):
    return 1.0 * (x > 0)

In [ ]:
# 評估處理 (ReLU 函數版)
from sklearn.metrics import accuracy_score

def evaluate2(x_test, y_test, y_test_one, V, W):
    b1_test = np.insert(ReLU(x_test @ V), 0, 1, axis=1)
    yp_test_one = softmax(b1_test @ W)
    yp_test = np.argmax(yp_test_one, axis=1)
    loss = cross_entropy(y_test_one, yp_test_one)
    score = accuracy_score(y_test, yp_test)
    return score, loss

In [ ]:
# 將 Sigmoid 更換為 LeRU 函數)
while epoch < nb_epoch:
    
    # 挑選出小批量
    index, next_flag = indexes.next_index()
    x, yt = x_train[index], y_train_one[index]

    # 預測值計算 (前饋式) 
    a = x @ V
    b = ReLU(a)                       # 激活函數從 Sigmoid 變成 ReLU
    b1 = np.insert(b, 0, 1, axis=1)   # 加入虛擬變數 
    u = b1 @ W
    yp = softmax(u)
    
    # 誤差計算 
    yd = yp - yt
    bd = step(a) * (yd @ W[1:].T)     # 從 Sigmoid 的微分變成 ReLU 的微分

    # 梯度計算
    W = W - alpha * (b1.T @ yd) / B
    V = V - alpha * (x.T @ bd) / B

    if next_flag: # 1 epoch 結束後記錄
        score, loss = evaluate2(
            x_test, y_test, y_test_one, V, W)
        history3 = np.vstack((history3, 
            np.array([epoch, loss, score])))
#        print("epoch = %d loss = %f score = %f" 
#             % (epoch, loss, score))
        epoch = epoch + 1

In [ ]:
#損失函數值與準確率確認
print('初始狀態: 損失函數:%f 準確率:%f' 
         % (history3[0,1], history3[0,2]))
print('最終狀態: 損失函數:%f 準確率:%f' 
         % (history3[-1,1], history3[-1,2]))

# 學習曲線 (損失函數值)
plt.plot(history3[:,0], history3[:,1])
plt.ylim(0,2.5)
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.show()

# 學習曲線 (準確率)
plt.plot(history3[:,0], history3[:,2])
plt.ylim(0,1)
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.show()

# 選出 20 筆資料
import matplotlib.pyplot as plt
N = 20
np.random.seed(12)
indexes = np.random.choice(y_test.shape[0], N, replace=False)

In [ ]:
# x_org 選取
x_selected = x_test[indexes]
y_selected = np.squeeze(np.array([y_test.iloc[indexes]]), axis=0)

# 預測值計算
b1_test = np.insert(ReLU(x_selected @ V), 0, 1, axis=1)
yp_test_one = softmax(b1_test @ W)
yp_test = np.argmax(yp_test_one, axis=1)

# 畫出圖形
plt.figure(figsize=(10, 3))
for i in range(N):
    ax = plt.subplot(2, N//2, i + 1)
    plt.imshow(x_selected[i,1:].reshape(28, 28),cmap='gray_r')
    ax.set_title('%d:%d' % (y_selected[i], yp_test[i]),fontsize=14 )
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()